<h1 style="font-weight: bold;">Titanic Survival With R</h1>


## 1) **Import libraries**

load the libraries to be used in the notebook


In [183]:
library(magrittr)
library(tidyverse)
library(tidymodels)
library(psych)
library(R6)
library(DescTools)


Attaching package: ‘DescTools’


The following objects are masked from ‘package:psych’:

    AUC, ICC, SD




## 2) **Setting notebook**


In [150]:
options(indent = 4)


## 3) **Load the data**

<p>Load the csv file to pandas dataframe</p>


In [151]:
folder_path <- NULL

if (dir.exists("data/")) {
  folder_path <- "data"
} else {
  stop("Unable find the data")
}


train_path <- file.path(folder_path, "train.csv")
test_path <- file.path(folder_path, "test.csv")


In [152]:
# Set the PassengerId column as index column.
index_col <- "PassengerId"

# Load the train and test data frame.
train_df <- read_csv(train_path)
test_df <- read_csv(test_path)


Rows: 891 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): Name, Sex, Ticket, Cabin, Embarked
dbl (7): PassengerId, Survived, Pclass, Age, SibSp, Parch, Fare

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 418 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): Name, Sex, Ticket, Cabin, Embarked
dbl (6): PassengerId, Pclass, Age, SibSp, Parch, Fare

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [153]:
index_col <- "PassengerId"

# Remove the PassengerId column for data index
train_df <- train_df %>% select(-any_of(index_col))
test_df <- test_df %>% select(-any_of(index_col))


## 4) **Data statistics**

<p>Analyse train and test dataframe</p>


### i) Rows in train data

<p>View train dataframe top and bottom rows</p>


In [154]:
# First five rows in train data.
train_df %>% head()


Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NA,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NA,S
1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NA,S
0,3,"Moran, Mr. James",male,NA,0,0,330877,8.4583,NA,Q


In [155]:
# First five rows in test data.
train_df %>% tail()


Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
0,3,"Rice, Mrs. William (Margaret Norton)",female,39,0,5,382652,29.125,NA,Q
0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13.000,NA,S
1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.000,B42,S
0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NA,1,2,W./C. 6607,23.450,NA,S
1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.000,C148,C
0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.750,NA,Q


### ii) Rows in test data

<p>View test dataframe top and bottom rows</p>


In [156]:
# First five rows in test data.
test_df %>% head()


Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NA,Q
3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NA,S
2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NA,Q
3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NA,S
3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NA,S
3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NA,S


In [157]:
# Last five rows in test data
test_df %>% tail()


Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
3,"Henriksson, Miss. Jenny Lovisa",female,28.0,0,0,347086,7.7750,NA,S
3,"Spector, Mr. Woolf",male,NA,0,0,A.5. 3236,8.0500,NA,S
1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NA,S
3,"Ware, Mr. Frederick",male,NA,0,0,359309,8.0500,NA,S
3,"Peter, Master. Michael J",male,NA,1,1,2668,22.3583,NA,C


### iii) Data description

<p>Get the know the datasets</p>


#### a) Data shapes


In [158]:
# Train data frame shape.
train_shape <- train_df %>% dim()
display_shape <- sprintf(
  "Train shape:\n\trows: %s\n\tcolumns: %s",
  train_shape[1],
  train_shape[2]
)
cat(display_shape)


Train shape:
	rows: 891
	columns: 11

In [159]:
# Test data frame shape.
test_shape <- test_df %>% dim()
display_shape <- sprintf(
  "Test shape:\n\trows: %s\n\tcolumns: %s",
  test_shape[1],
  test_shape[2]
)
cat(display_shape)


Test shape:
	rows: 418
	columns: 10

#### b) Data column names


In [160]:
# Formats the vectors
format_vec <- function(vec) {
  cat("[\n")
  for (i in vec) {
    cat("  ", i, ",\n", sep = "")
  }
  cat("]")
}


In [161]:
columns <- train_df %>% colnames()
format_vec(columns)


[
  Survived,
  Pclass,
  Name,
  Sex,
  Age,
  SibSp,
  Parch,
  Ticket,
  Fare,
  Cabin,
  Embarked,
]

#### c) Data information


In [162]:
# Train data information.
train_df %>% str()


tibble [891 × 11] (S3: tbl_df/tbl/data.frame)
 $ Survived: num [1:891] 0 1 1 1 0 0 0 0 1 1 ...
 $ Pclass  : num [1:891] 3 1 3 1 3 3 1 3 3 2 ...
 $ Name    : chr [1:891] "Braund, Mr. Owen Harris" "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" "Heikkinen, Miss. Laina" "Futrelle, Mrs. Jacques Heath (Lily May Peel)" ...
 $ Sex     : chr [1:891] "male" "female" "female" "female" ...
 $ Age     : num [1:891] 22 38 26 35 35 NA 54 2 27 14 ...
 $ SibSp   : num [1:891] 1 1 0 1 0 0 0 3 0 1 ...
 $ Parch   : num [1:891] 0 0 0 0 0 0 0 1 2 0 ...
 $ Ticket  : chr [1:891] "A/5 21171" "PC 17599" "STON/O2. 3101282" "113803" ...
 $ Fare    : num [1:891] 7.25 71.28 7.92 53.1 8.05 ...
 $ Cabin   : chr [1:891] NA "C85" NA "C123" ...
 $ Embarked: chr [1:891] "S" "C" "S" "S" ...


In [163]:
# Test data information
test_df %>% str()


tibble [418 × 10] (S3: tbl_df/tbl/data.frame)
 $ Pclass  : num [1:418] 3 3 2 3 3 3 3 2 3 3 ...
 $ Name    : chr [1:418] "Kelly, Mr. James" "Wilkes, Mrs. James (Ellen Needs)" "Myles, Mr. Thomas Francis" "Wirz, Mr. Albert" ...
 $ Sex     : chr [1:418] "male" "female" "male" "male" ...
 $ Age     : num [1:418] 34.5 47 62 27 22 14 30 26 18 21 ...
 $ SibSp   : num [1:418] 0 1 0 0 1 0 0 1 0 2 ...
 $ Parch   : num [1:418] 0 0 0 0 1 0 0 1 0 0 ...
 $ Ticket  : chr [1:418] "330911" "363272" "240276" "315154" ...
 $ Fare    : num [1:418] 7.83 7 9.69 8.66 12.29 ...
 $ Cabin   : chr [1:418] NA NA NA NA ...
 $ Embarked: chr [1:418] "Q" "S" "Q" "S" ...


#### d) Describing the data


In [164]:
n_unique <- function(x) length(unique(x))
is_numeric_vec <- function(x, threshold_num = 10) n_unique(x) > threshold_num

desc <- function(data, display_stat = "numeric", threshold_num = 10) {
  if (display_stat == "numeric") {
    numeric_result <- data %>%
      select(where(is.numeric)) %>%
      select(where(is_numeric_vec)) %>%
      describe() %>%
      t()

    return(numeric_result)
  } else if (display_stat == "character") {
    cat_result <- data %>%
      select(where(
        function(vec) {
          is.character(vec) || is.factor(vec) || !is_numeric_vec(
            vec,
            threshold_num = threshold_num
          )
        }
      )) %>%
      lapply(function(vec) {
        c((!(vec %>% is.na())) %>% sum(), n_unique(vec))
      }) %>%
      data.frame()
    row.names(cat_result) <- c("n", "n_unique")

    return(cat_result %>% t())
  }
}


<p style="font-weight: bold; font-style: italic;">Description of numerical columns in train data</p>


In [165]:
train_df %>% desc()


,Age,Fare
vars,1.0000000,2.000000
n,714.0000000,891.000000
mean,29.6991176,32.204208
sd,14.5264973,49.693429
median,28.0000000,14.454200
trimmed,29.2692308,21.378716
mad,13.3434000,10.236167
min,0.4200000,0.000000
max,80.0000000,512.329200
range,79.5800000,512.329200


<p style="font-weight: bold; font-style: italic;">Description of categorical columns in train data</p>


In [166]:
train_df %>% desc(display_stat = "character")


,n,n_unique
Survived,891,2
Pclass,891,3
Name,891,891
Sex,891,2
SibSp,891,7
Parch,891,7
Ticket,891,681
Cabin,204,148
Embarked,889,4


## 5) **Data split**

<p>Separate target from the data and then split the data into train and validation</p>


In [167]:
# Initial split 🪓
init_split <- train_df %>% initial_split(prop = 3 / 4)


In [168]:
# 75 percent will be for train data
train_data <- init_split %>% training()

# 25 percent will be for test data.
test_data <- init_split %>% testing()

# View the shape of the data.
train_dim <- train_data %>% dim()
test_dim <- test_data %>% dim()


cat(sprintf(
  "Train sample:\n  Rows: %d\n  Columns: %d\n",
  train_dim[1], train_dim[2]
))

cat("\n")

cat(sprintf(
  "Test sample:\n  Rows: %d\n  Columns: %d\n",
  test_dim[1], test_dim[2]
))

Train sample:
  Rows: 668
  Columns: 11

Test sample:
  Rows: 223
  Columns: 11


## 6) **Data cleaning**

<p>Modifying or removing data that is inaccurate, duplicate, incomplete, incorrectly formatted, or corrupted</p>


### i) Check the missing values (NaN)

<p>Search for missing value in the dataframes</p>


In [210]:
handle_nan <- R6Class(
  "handle_nan",
  private = list(
    train_df = NULL,
    test_df = NULL,
    strategy = NULL,
    fill_value_to_na = function(column_name, strategy) {
      private$train_df <- private$train_df %>%
        mutate(
          "{column_name}" := replace_na(
            .data[[column_name]],
            private$map_value_to_fill(.data[[column_name]], strategy)
          )
        )
    },
    map_value_to_fill = function(vec, strategy) {
      value <- NULL

      if (strategy %in% c("mean", "median", "mode")) {
        value <- switch(strategy,
          mean = mean(vec, na.rm = TRUE) %>% round(digits = 2),
          median = median(vec, na.rm = TRUE) %>% round(digits = 2),
          mode = Mode(vec)
        )
      } else {
        value <- strategy
      }

      value
    }
  ),
  public = list(
    initialize = function(train_df = NA, test_df = NA, strategy = NA) {
      private$train_df <- train_df
      private$test_df <- test_df
      private$strategy <- strategy
    },
    handle = function() {
      get_names <- names(private$strategy)

      for (i in seq_along(private$strategy)) {
        private$fill_value_to_na(
          get_names[i],
          private$strategy[[i]]
        )
      }

      private$train_df
    }
  )
)

fill_data <- handle_nan$new(
  train_data,
  test_data,
  strategy = list(Age = "mode")
)

fill_data$handle()


Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
0,3,"Hanna, Mr. Mansour",male,23.5,0,0,2693,7.2292,NA,C
0,2,"Fynney, Mr. Joseph J",male,35.0,0,0,239865,26.0000,NA,S
0,3,"Adahl, Mr. Mauritz Nils Martin",male,30.0,0,0,C 7076,7.2500,NA,S
0,3,"Shorney, Mr. Charles Joseph",male,NA,0,0,374910,8.0500,NA,S
0,3,"Sadlier, Mr. Matthew",male,NA,0,0,367655,7.7292,NA,Q
0,3,"Tomlin, Mr. Ernest Portage",male,30.5,0,0,364499,8.0500,NA,S
0,3,"Ling, Mr. Lee",male,28.0,0,0,1601,56.4958,NA,S
0,3,"Radeff, Mr. Alexander",male,NA,0,0,349223,7.8958,NA,S
1,2,"Mallet, Master. Andre",male,1.0,0,2,S.C./PARIS 2079,37.0042,NA,C


In [190]:
train_data[["Age"]]

[1] 23.50 35.00 30.00    NA    NA 30.50 28.00    NA  1.00 30.00  8.00 70.00
 [13] 21.00  1.00 23.00    NA 80.00 45.00 18.00 28.00 24.00 27.00 22.00 40.00
 [25] 48.00    NA 34.00 42.00  2.00    NA  1.00    NA    NA    NA 57.00 25.00
 [37] 37.00 22.00 27.00    NA 48.00 23.00 34.00 66.00 31.00 36.00 24.00    NA
 [49] 21.00    NA    NA 24.00 17.00 21.00 16.00    NA    NA 30.00  0.83 28.00
 [61] 24.00 39.00    NA 27.00 22.00  5.00 21.00 42.00    NA 18.00 24.50 45.00
 [73] 25.00 23.00 24.00    NA 50.00    NA 23.00  3.00  0.92 21.00    NA 18.00
 [85]    NA 30.00 28.00    NA  0.42 36.00 16.00 29.00 28.00 56.00 36.00 26.00
 [97] 22.00  1.00 44.00 54.00 29.00 16.00 26.00    NA 19.00    NA 30.00    NA
[109]    NA 42.00 26.00 32.00    NA 39.00 45.00 28.00    NA  5.00 51.00 55.00
[121] 60.00 48.00 42.00 47.00    NA 40.00    NA    NA 32.00 19.00 25.00    NA
[133] 20.00 39.00 29.00    NA 30.00 32.00 16.00 31.00 42.00 32.00 24.00 37.00
[145] 18.00    NA 43.00 22.00 35.00 22.00 32.50 17.00 25.00 24.00    NA 39.00
[157] 23.00    NA 20.00 19.00 39.00    NA    NA 19.00  8.00    NA 22.00 24.00
[169] 23.00 38.00 24.00 52.00 23.00  6.00 44.00 25.00 35.00    NA 29.00    NA
[181]    NA 43.00    NA 28.00    NA 35.00 30.00    NA 40.00    NA 58.00 28.00
[193] 24.00 24.00 39.00    NA 54.00 27.00 25.00 32.00    NA 25.00 36.00 36.00
[205] 30.50    NA 30.00 55.00 21.00 46.00 21.00 20.00    NA 22.00 23.00 35.00
[217]    NA 71.00 45.00 42.00 18.00    NA 34.00 29.00 47.00 28.00    NA 17.00
[229] 32.50 71.00 28.00 21.00 50.00 24.00    NA 39.00 14.00    NA    NA 62.00
[241] 35.00  2.00 20.00    NA 37.00  3.00 42.00 33.00 31.00    NA 26.00 17.00
[253] 26.00 28.50 33.00 29.00    NA 26.00 65.00 41.00 28.00    NA    NA 29.00
[265]    NA 30.00 21.00 25.00 24.00 30.00 17.00    NA 15.00 31.00 15.00 24.00
[277]  2.00 10.00    NA 30.00 48.00  3.00 22.00 19.00    NA 34.00 22.00  4.00
[289] 50.00  1.00 33.00  0.83 20.00 32.00 22.00 20.00 11.00 74.00    NA 31.00
[301] 46.00 16.00 16.00 36.00 48.00 22.00 16.00 39.00    NA 70.50    NA 51.00
[313]  4.00 39.00    NA 28.00 18.00 32.00 35.00 51.00 31.00 19.00 45.00 10.00
[325] 44.00 22.00  2.00 61.00 21.00 34.00 36.00 54.00 33.00 56.00 42.00 27.00
[337] 31.00 18.00 11.00    NA 18.00 33.00 30.00    NA 40.00 20.00 41.00 64.00
[349] 44.00  4.00 16.00 45.50  3.00 32.00  4.00    NA    NA 40.00 36.00 14.00
[361] 27.00 31.00 29.00 54.00 25.00 41.00    NA 18.00 13.00 27.00 29.00 35.00
[373] 17.00 25.00  8.00 48.00  4.00 25.00 52.00  2.00 30.00 21.00 47.00  6.00
[385]  7.00    NA 36.00 17.00 40.00    NA 48.00    NA 61.00 27.00 24.00 18.00
[397] 32.00 27.00    NA 19.00 31.00 27.00 28.00 30.00 50.00 20.00 33.00 24.00
[409]    NA 59.00 34.00 16.00 19.00 34.00    NA 27.00 29.00 12.00    NA 52.00
[421]    NA 38.00    NA  2.00  3.00    NA 30.00 26.00 36.00 45.00 32.00 36.00
[433] 14.00 19.00 19.00 25.00 22.00    NA 46.00 13.00    NA    NA 40.00    NA
[445] 58.00 47.00 35.00 48.00 61.00 26.00 45.00 27.00 19.00 34.00 24.00 36.50
[457] 45.00 21.00 40.00 14.00 60.00 16.00 24.00 35.00 32.00 51.00 38.00 29.00
[469]    NA 39.00 56.00 16.00 36.00 15.00 31.00 31.00 33.00  8.00 42.00 19.00
[481]    NA    NA 42.00 25.00    NA 28.00 47.00  7.00 49.00 28.00 57.00 51.00
[493] 43.00 14.00  9.00    NA 18.00    NA 38.00 38.00  1.00    NA 24.00 54.00
[505] 23.00 36.00 18.00    NA    NA 20.00 32.00 28.00 33.00 34.50 19.00 24.00
[517] 17.00 55.50    NA 42.00 19.00 33.00 65.00    NA    NA    NA 36.00  6.00
[529] 28.00 40.00 24.00 60.00  0.75 49.00 29.00 27.00 25.00 28.00 58.00 35.00
[541] 41.00    NA 47.00 64.00    NA 27.00 31.00 22.00 26.00 11.00 27.00 49.00
[553] 21.00 27.00 15.00 21.00    NA 25.00 36.00    NA    NA 33.00  5.00 20.00
[565]    NA 32.00 18.00 19.00 19.00 51.00 36.00 20.00 44.00    NA 19.00 22.00
[577] 37.00 70.00 28.00 17.00    NA 41.00    NA 60.00  2.00  7.00 25.00 30.00
[589] 45.00    NA 30.00 18.00 14.00 31.00    NA 19.00    NA 18.00 25.00 21.00
[601] 32.00    NA 23.00 18.00 33.00 50.00 17.00 40.00 43.00 35.00 58.00 44.00
[613]  5.00  9.00 28.00 

In [207]:
x <- c(7, 5, 3)
Mode(c(0:8, 1))

[1] 1
attr(,"freq")
[1] 2

Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”


ERROR: Error in "a":"z": NA/NaN argument


In [171]:
fill <- function(data, col, strata) {
  data %>% mutate("{col}" := replace_na(
    .data[[col]], 
    switch(
      strata,
      mean = mean(.data[[col]])
      )
    ))
}

train_data %>% fill("Age", "mean")

Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
0,3,"Hanna, Mr. Mansour",male,23.5,0,0,2693,7.2292,NA,C
0,2,"Fynney, Mr. Joseph J",male,35.0,0,0,239865,26.0000,NA,S
0,3,"Adahl, Mr. Mauritz Nils Martin",male,30.0,0,0,C 7076,7.2500,NA,S
0,3,"Shorney, Mr. Charles Joseph",male,NA,0,0,374910,8.0500,NA,S
0,3,"Sadlier, Mr. Matthew",male,NA,0,0,367655,7.7292,NA,Q
0,3,"Tomlin, Mr. Ernest Portage",male,30.5,0,0,364499,8.0500,NA,S
0,3,"Ling, Mr. Lee",male,28.0,0,0,1601,56.4958,NA,S
0,3,"Radeff, Mr. Alexander",male,NA,0,0,349223,7.8958,NA,S
1,2,"Mallet, Master. Andre",male,1.0,0,2,S.C./PARIS 2079,37.0042,NA,C


In [172]:
train_data %>% fill

ERROR: [1m[33mError[39m:[22m
[33m![39m Failed to evaluate glue component {col}
[1mCaused by error:[22m
[33m![39m argument "col" is missing, with no default


In [ ]:
def plot_nan(data: pd.DataFrame, name: str | None = None, only_nan: bool = True, use_percent=False) -> None:
    nan_df = check_n_nan(data, only_nan)
    nan_df['percent_nan'] = nan_df['percent_nan'].str.replace(
        '%', '').astype(float).round().astype(int)

    radius = 5
    y_name = 'percent_nan' if use_percent else 'total_nan'
    startup_title = 'Percentage' if use_percent else 'Number'
    y_title = f'{startup_title} of missing values'
    y_name = alt.Y(y_name).title(y_title)
    name = name if name else 'a dataset'
    title = f'{startup_title} Of Missing values In {name.title()}'
    subtitle = f'A chart represents missing data in {name}'

    base = alt.Chart(
        nan_df,
        title=alt.Title(
            title,
            subtitle=subtitle,
            anchor='start',
        )
    ).mark_bar(
        cornerRadiusTopLeft=radius,
        cornerRadiusTopRight=radius
    ).encode(
        x=alt.X('columns').title('Columns names'),
        y=y_name,
        color=alt.Color('columns').scale(
            scheme='lightgreyred',
            reverse=True
        ).legend(orient='right'),
        tooltip=["columns", "total_nan", 'percent_nan'],
        text='columns'
    ).configure_axis(
        grid=False
    ).configure_view(
        stroke=None
    )

    return base

ERROR: Error in parse(text = input): <text>:1:5: unexpected symbol
1: def plot_nan
        ^


#### a) Observing missing data in train and validation data


##### One) X train dataset


In [ ]:
check_n_nan(train_x)


In [ ]:
plot_nan(
  train_x,
  name = "x train dataset",
  only_nan = False
)


**conclusion:** X train data sample contains missing data of 21.11%(141) of Age column, 78.29%(523) of cabin column and 0.3%(2) of Embarked column are the amount of missing data in the dataset from the data size of 668.


##### Two) X validation dataset


In [ ]:
check_n_nan(valid_x)


In [ ]:
plot_nan(
  valid_x,
  name = "x validation dataset",
  only_nan = False,
  use_percent = True
)


**conclusion:** In X validation data sample there are 16.14%(36) of Age column and 73.54%(164) of cabin column are the amount of missing data in the dataset from the data size of 223.


#### b) Observing missing data in test dataset


In [ ]:
check_n_nan(test_df)


In [ ]:
plot_nan(
  test_df,
  name = "test dataset",
  only_nan = False,
  use_percent = False
)


**conclusion:** The test data frame contains missing data of 20.57%(86) of Age column, 78.23%(327) of cabin column and 0.24%(1) of Fare column are the amount of missing data in the dataset from the data size of 418.


### ii) Handling the missing values (NaN)

<p>Filling missing value in the dataframes</p>


In [ ]:
class FillNaN:
    def __init__(self, dfs: List[pd.DataFrame], fill_with: Dict[str, Any]) -> None:
        self.dfs = dfs
        self.fill_with = fill_with

    @property
    def fill(self) -> List[pd.DataFrame]:
        """
        Fills missing values in the data frames.
        :return: A tuple of data frames
        """
        lst = []
        for df in self.dfs:
            for col, strat in self.fill_with.items():
                df = self.strategy(df, strat=strat, col_name=col)
            lst.append(df)
        return lst

    @staticmethod
    def strategy(df: pd.DataFrame, strat: str, col_name: str) -> pd.DataFrame:
        match strat:
            case 'mean':
                df[col_name] = df[col_name].fillna(df[col_name].mean())
            case 'mode':
                df[col_name] = df[col_name].fillna(df[col_name].mode()[0])
            case 'min':
                df[col_name] = df[col_name].fillna(df[col_name].min())
            case 'max':
                df[col_name] = df[col_name].fillna(df[col_name].max())
            case 'median':
                df[col_name] = df[col_name].fillna(df[col_name].median())
            case 'bfill':
                df[col_name] = df[col_name].bfill()
            case 'ffill':
                df[col_name] = df[col_name].ffill()
            case "drop":
                df = df.drop(col_name, axis=1)
            case _:
                raise ValueError(
                    f"Ooh, strategy has received invalid value `{strat}` for strat")
        return df

Filling the missing value in age with mean, Fare with ffill, Embarked with mode and Cabin with drop.


In [ ]:
fill = FillNaN(
    [train_x, valid_x, test_df],
    {"Age": "mean",
     "Fare": "ffill",
     "Embarked": "mode",
     "Cabin": "drop"})

In [ ]:
clean_train_x, clean_valid_x, clean_test_df = fill.fill

Check if there still any missing values in the data.


**Clean train x**


In [ ]:
check_n_nan(clean_train_x, only_nan = False)


**Clean valid x**


In [ ]:
check_n_nan(clean_valid_x, only_nan = False)


**Clean test df**


In [ ]:
check_n_nan(clean_test_df, only_nan = False)


### iii) Handling the categorical columns

<p>Encode categorical column to integer and drop high cardinality columns in the dataframes</p>


In [ ]:
class CategoryEncoder:
    def __init__(
            self,
            datasets: List[pd.DataFrame],
            handle_with: Dict[str, str],
    ) -> None:
        self.datasets = datasets
        self.handle_with = handle_with

    def strategy(self, df: pd.DataFrame, col: str, strat: str) -> pd.DataFrame:
        match strat:
            case 'label_encode':
                series = df[col].astype('category').cat.codes
                df[col] = series
            case 'one_hot_encode':
                data = pd.get_dummies(
                    df[col],
                    columns=list(df[col].unique())
                ).astype(int)
                df = pd.concat([df, data], axis=1)
                df = df.drop(col, axis=1)
            case 'drop':
                df = df.drop(col, axis=1)
            case _:
                raise ValueError()

        return df

    def encode(self) -> List[pd.DataFrame]:
        """
        Encode categorical columns to integer.
        :return: A tuple of data frames
        """

        datasets = []

        for dataset in self.datasets:
            for col, strat in self.handle_with.items():
                dataset = self.strategy(dataset, col, strat)
            datasets.append(dataset)

        return datasets

In [ ]:
# First five rows from the clean train x.
clean_train_x.head()


In [ ]:
# Exclude the int and float from the data.
cat_df = clean_train_x.select_dtypes(exclude=[int, float])
cat_df.head()

In [ ]:
# Fetch the categorical columns.
cat_columns <- list(cat_df.columns)
cat_columns


Check for columns with high cardinality column in the data.


In [ ]:
clean_train_x[cat_columns].nunique()

In [ ]:
# Instantiate the CategoryEncoder
encoder = CategoryEncoder(
    [clean_train_x, clean_valid_x, clean_test_df],
    {
        "Sex": "label_encode",
        "Embarked": "one_hot_encode",
        "Name": "drop",
        "Ticket": "drop"
    }
)

# Encode the column
num_train_x, num_valid_x, num_test_df = encoder.encode()

In [ ]:
num_train_x.dtypes


## 7) **Data visualisation**

<p>Represeting the data in graphical way like making chart</p>


In [ ]:
visual_df = pd.concat([clean_train_x, clean_valid_x])
visual_df['Survived'] = np.concatenate([train_y, valid_y])
visual_df.head()

In [ ]:
alt.renderers.enable("html")


class Graph:
    def __init__(
        self, data: pd.DataFrame, map_values: Optional[Dict[str, Dict[str, Any]]] = None
    ) -> None:
        data = data.copy()

        if map_values:
            for key, value in map_values.items():
                data[key] = data[key].map(lambda col: value[col])

        self.__data = data

    def setting_colors(
        self, x: str, color: List[str] | str
    ) -> Tuple[pd.DataFrame, alt.Color]:

        # Color map.
        if isinstance(color, list):
            alt_color = (
                alt.Color(x).scale(range=color, reverse=True).legend(orient="right")
            )
        else:
            alt_color = (
                alt.Color(x).scale(scheme=color, reverse=True).legend(orient="right")
            )
        return alt_color

    def col_bar_char(
        self,
        x_column: str,
        group_by: str,
        color: List | str = "paired",
        width: int = 200,
        height: int = 200,
        title: Optional[str] = None,
        subtitle: Optional[str] = None,
        x_title: str | None = None,
        y_title: str | None = None,
    ):
        alt_color = self.setting_colors(x_column, color=color)

        data = self.__data

        x_title = x_title.title() if x_title else x_column.title()
        y_title = y_title.title() if y_title else "Number of Passengers"

        source = data.groupby(group_by)[x_column].value_counts().reset_index()

        grouped_bars = (
            alt.Chart(source)
            .mark_bar(cornerRadius=10)
            .encode(
                x=alt.X(x_column, title=x_title),
                y=alt.Y("count:Q", title=y_title),
                color=alt_color,
                column=group_by,
            )
            .encode(
                tooltip=[
                    alt.Tooltip(x_column, title=x_title),
                    alt.Tooltip("count", title=y_title),
                ],
            )
            .properties(
                width=width,
                height=height,
                title=alt.Title(
                    title if title else "",
                    fontSize=18,
                    subtitle=subtitle if subtitle else "",
                    subtitleFontSize=12,
                    # dy=-10,
                ),
            )
        )

        return grouped_bars

    def bar_chart(
        self,
        x_column: str,
        color: List | str = "paired",
        width: int = 200,
        height: int = 200,
        title: Optional[str] = None,
        subtitle: Optional[str] = None,
        x_title: str | None = None,
        y_title: str | None = None,
    ) -> alt.ChartDataType:  # Make copy then change column data to str

        alt_color = self.setting_colors(x_column, color=color)

        data = self.__data

        x_title = x_title.title() if x_title else x_column.title()
        y_title = y_title.title() if y_title else "Number of Passengers"

        # Return the alt chart
        bar_layer = (
            alt.Chart(data)
            .mark_bar(cornerRadius=10)
            .encode(
                x=alt.X(
                    x_column,
                    title=x_title,
                    axis=alt.Axis(grid=False, labelFontWeight="bold"),
                ),
                y=alt.Y(
                    "count()",
                    title=y_title,
                    axis=alt.Axis(grid=False, labelFontWeight="bold"),
                ),
                color=alt_color.legend(orient="right"),
                tooltip=[
                    alt.Tooltip(x_column, title=x_title),
                    alt.Tooltip("count()", title=y_title),
                ],
            )
            .properties(
                width=width,
                height=height,
                title=alt.Title(
                    title if title else "",
                    fontSize=18,
                    subtitle=subtitle if subtitle else "",
                    subtitleFontSize=12,
                    # dy=-10,
                ),
            )
        )

        text_layer = (
            alt.Chart(data)
            .mark_text(
                align="center",
                baseline="bottom",  # Align text to the bottom of its bounding box
                dy=-5,  # Adjust vertical position (negative for above the bar)
                fontSize=12,
                fontWeight="bold",
            )
            .encode(x=x_column, y="count()", text=alt.Text("count()"))
        )

        return bar_layer + text_layer  # .configure_view(

    def donut_chart(
        self,
        x: str,
        inner_radius: int = 50,
        color: List | str = "paired",
        width: int = 200,
        height: int = 200,
    ) -> alt.ChartDataType:

        data = self.__data

        value_counts = data[x].value_counts().reset_index()
        value_counts["count"] = (value_counts["count"] / data.shape[0]).round(3)

        donut_chart = (
            alt.Chart(value_counts)
            .mark_arc(innerRadius=inner_radius)
            .encode(
                theta="count",
                color=f"{x}:N",
                tooltip=[
                    alt.Tooltip(f"{x}".title()),
                    alt.Tooltip("count", title="Percentage"),
                ],
            )
            .properties(
                width=width,
                height=height,
            )
        )
        return donut_chart

    def scatter_chart(
        self,
        x: str,
        y: str,
        color_with: Optional[str] = None,
        width: int = 300,
        height: int = 300,
        title: Optional[str] = None,
        subtitle: Optional[str] = None,
        color: List[str] | str = "dark2",
    ):
        alt_color = None

        if color_with:
            alt_color = self.setting_colors(color_with, color=color)

        tooltips = (
            [alt.Tooltip(x), alt.Tooltip(y), alt.Tooltip(color_with)]
            if color_with
            else [alt.Tooltip(x), alt.Tooltip(y)]
        )

        encode_param = (
            dict(x="%s:Q" % x, y="%s:Q" % y, color=alt_color, tooltip=tooltips)
            if color_with
            else dict(x="%s:Q" % x, y="%s:Q" % y, tooltip=tooltips)
        )

        data = self.__data

        # Scatter Plot
        points = (
            alt.Chart(data)
            .mark_point()
            .encode(**encode_param)
            .properties(
                width=width,
                height=height,
                title=alt.Title(
                    title if title else "",
                    fontSize=18,
                    subtitle=subtitle if subtitle else "",
                    subtitleFontSize=12,
                ),
            )
        )

        points = points.configure_view(stroke=None)

        return points

    def concat(
        self,
        *args,
        use: Literal["vconcat", "hconcat"] = "vconcat",
        show_stroke: bool = False,
    ):
        if use == "vconcat":
            if show_stroke:
                return alt.vconcat(*args)
            return alt.vconcat(*args).configure_view(stroke=None)
        elif use == "hconcat":
            if show_stroke:
                return alt.hconcat(*args)
            return alt.hconcat(*args).configure_view(stroke=None)
        else:
            raise ValueError("Received invalid value `%s`" % use)


graph = Graph(
    visual_df,
    map_values={
        "Sex": {0: "Female (0)", 1: "Male (1)"},
        "Survived": {0: "Didn't survive", 1: "survived"},
    },
)
graph.bar_chart("Embarked")
donut = graph.donut_chart("Embarked")
col_bar = graph.scatter_chart(
    "Age", "Fare", color_with="Survived", color=["red", "blue"]
)
col_bar

# hconcat = graph.concat(bar, donut, use="hconcat", show_stroke=True)
# graph.concat(hconcat, col_bar, use="vconcat")
# col_bar

In [ ]:
x = clean_train_x['Embarked'].value_counts().reset_index()
x["count"] = (x["count"] / clean_train_x.shape[0]).round(3)